In [165]:
# Jupyer Notebook Theme: jt -t monokai -f fira -fs 10 -nf ptsans -nfs 11 -N -kl -cursw 2 -cursc r -cellw 95% -T
# Check python version
import sys
cdPyVer = 0x600
cdSysVer = sys.version[:5]
print("%s%s" %("My python version is: ", cdSysVer)) # strangely it works only on 3.7.4 and not 3.7.6

# Import libraries
import pandas as pd
import numpy as np
import warnings


# Visualisation Libraries
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.gridspec import GridSpec
import seaborn as sns
plt.style.use('seaborn-dark') # use this theme as you are using dark theme in the notebook

# SK Learn Libraries
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier


# Diplay maximum colums, rows, column_width and float format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.float_format', lambda x: '%.1f' % x)

warnings.filterwarnings('ignore')

My python version is: 3.9.7


In [166]:
df = pd.read_csv('../datasets/titanic_data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Start Pipeline

In [167]:
# Drop columns not needed!
df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

#### Step1: Train-Test-Split

In [168]:
#### Step1: Train-Test-Split
X = df.drop(columns = ['Survived']) # All columns except survived is an input feature
y = df['Survived'] # Dependent variable

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Ensure training dataset has only 70% of original X data
print(X_train.shape[0] / X.shape[0])

# Ensure testing dataset has only 30% of original X data
print(X_test.shape[0] / X.shape[0])

0.6992143658810326
0.30078563411896747


#### Step2: Inputation, One-Hot Encoding, Scaling, Feature Selection, Model Training

In [169]:
# 177 null values for Age and 2 for Embarked
# Age is index 2
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [170]:
# Age is index 2
# Embarked is index 6
X_train.head(1)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
510,3,male,29.0,0,0,7.75,Q


In [171]:
#### Inputation transformer
trf1 = ColumnTransformer([
    ('inpute_age', SimpleImputer(), [2]), #Input mean for age
    ('input_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder = 'passthrough')

In [172]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse=False, handle_unknown='ignore'), [1,6]) # Sex is index 1, Embarked is index 6
], remainder='passthrough')

In [173]:
# Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,8))
])

In [174]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [175]:
# train the model
trf5 = DecisionTreeClassifier()

#### Step3: Create Pipeline

In [176]:
pipe = Pipeline([
    ('simple_inputter',trf1),
    ('one_hot_encoding',trf2),
    ('standard_scaler',trf3),
    ('feature_selection',trf4),
    ('decision_tree_model',trf5)
])

#### Step4: Train Model

In [177]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [178]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('simple_inputter',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('inpute_age', SimpleImputer(),
                                                  [2]),
                                                 ('input_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('one_hot_encoding',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('standard_scaler',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('feature_selection',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fd951460820>)),
                ('decision_tree_model', DecisionTreeClassifier())])

#### Step5: Predict on Test Set

In [179]:
y_pred = pipe.predict(X_test)

In [180]:
y_pred

array([0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0])

In [182]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6388129032258065

In [183]:
# Check out difference between what my model predicted on testing data and also what really happened!
actual_and_predicted_diff = pd.DataFrame({'Actual':y_test, 'Predicted': y_pred})
actual_and_predicted_diff.head(5)

,Actual,Predicted
871,1,0
716,1,1
754,1,0
685,0,1
548,0,0


In [185]:
print("Decision Tree Accuracy Score: ",pipe.score(X_test, y_test))

Decision Tree Accuracy Score:  0.6305970149253731


In [186]:
from sklearn.metrics import precision_score, recall_score, f1_score
print(f"Precision Score: {precision_score(y_test, y_pred)}")
print(f"Sensitivity/Recall Score: {recall_score(y_test, y_pred)}")
print(f"F1-Score Score: {f1_score(y_test, y_pred)}")

Precision Score: 0.5576923076923077
Sensitivity/Recall Score: 0.2761904761904762
F1-Score Score: 0.3694267515923567
